# Chaîne de Markov et génération de texte

Ce tutoriel présente l'utilisation d'une chaîne de Markov pour générer du texte.

In [ ]:
#Imports
import re

In [ ]:
#Variables
memory = 2

## Préparation des données

L'objectif étant de générer des mots (et non des caractères, comme on retrouve parfois dans la littérature), on va découper notre texte grâce à une expression régulière pour récupérer tous les mots.
On met également le texte en minuscule car les majuscules n'apportent (quasiment) aucune information. On pourrait également enlever les apostrophes ou autres signes de ponctuation pour améliorer le résultat final, et certains mots de liaisons/nombres...

In [ ]:
def most_common(lst):
    return max(set(lst), key=lst.count)

In [ ]:
def prepare():
    file = open("la-machine-a-explorer-le-temps--h-g-wells.txt", "r", encoding="utf8")
    text = file.read()
    words = re.findall(r"[\w]+", text.lower())
    return words

## Entraînement de l'algorithme

Pour apprendre à prédire des mots, on va regarder quel mot suit tel ou tel groupe de mots (la variable "memory" sert à dire combien de mots précédents on regarde).

On va ainsi construire, pour chaque groupe de mots précédents (ce sera notre clé), quels sont les mots suivants possibles.

In [ ]:
def train(words, memory=2):
    last_words = []
    dict_markov = {}
    for word in words:
        #Init first vector
        if len(last_words) < memory:
            last_words.append(word)
        else:
            key = tuple(last_words)
            #If key in dictionary, save new possible prediction
            if key in dict_markov:
                dict_markov[key].append(word)
            #Else, create a list to store next word predictions
            else:
                dict_markov[key] = [word]
        #Update last_words with the new word
        last_words.pop(0)
        last_words.append(word)
    return dict_markov

## Prédiction

Enfin, on écrit une fonction capable de prédire les mots suivants en s'appuyant sur notre "expérience".

Pour une clé, on va prédire le mot suivant puis l'ajouter à la clé et répéter le processus autant de fois que demandé.

In [ ]:
def predict(start_sentence, repeat=10, memory=2, dictionary={}):
    prediction = start_sentence
    last_words = re.findall(r"[\w]+", start_sentence.lower())
    if len(last_words) < memory:
            raise Exception("Not enough words for a prediction")
    else:
        last_words = last_words[-memory:]
    for i in range(repeat):
        key = tuple(last_words)
        if key in dictionary:
            #Choose prediction
            possibilities = dictionary[key]
            prediction += " "+most_common(possibilities)
            #Update last_words with the new word
            last_words.pop(0)
            last_words.append(most_common(possibilities))
    return prediction

## Mise en pratique

Regardez ce que prédit notre algorithme pour les 9 mots suivant les mots "l'intelligence artificielle", avec une mémoire de 2 mots :

In [ ]:
words = prepare()
dict_markov = train(words, memory=memory)
prediction = predict("L'intelligence artificielle", repeat=9, memory=memory, dictionary=dict_markov)
print(prediction)